In [ ]:
import io
import requests
import numpy as np
from PIL import Image, ImageDraw

localhost = False

if localhost:
    monailabel_url = "http://localhost:8000"
else:
    monailabel_url = "http://10.0.3.62:8000"


In [ ]:
# Get Monai Label server info

res = requests.get(
    f'{monailabel_url}/info'
)

res.raise_for_status()

res.json()


In [ ]:
# Select image from dataset

res = requests.post(
    f'{monailabel_url}/activelearning/random'
)

res.raise_for_status()

res.json()


In [ ]:
# Run Impartial inference

res = requests.post(
    f'{monailabel_url}/infer/impartial',
    params={"image": "image1", "output": "image"},
    # data={'params': '{"device": "cpu", "tile_size": [400, 400], "output": "default"}'}
)

res.raise_for_status()


In [ ]:
import io
import pickle

def normalize(xs):
    return (255.0 * (xs - xs.min())) / (xs.max() - xs.min())

def array_to_pil(xs):
    return Image.fromarray(np.uint8(normalize(xs)) , 'L')

im = pickle.load(io.BytesIO(res.content))

display(array_to_pil(im['0']['class_segmentation'][0][0]))

display(array_to_pil(im['0']['factors']['components'][0][0]))


In [ ]:
# Legacy. Draw results from deepedit sample app

contours = results['annotations'][0]['annotation']['elements'][0]['contours']

with Image.open("./Data/Vectra_2CH/nuclei/Easy1.png") as im:
    draw = ImageDraw.Draw(im)
    
    for c in contours:
        for p1, p2 in zip(c, c[1:]):
            draw.line((p1[0], p1[1], p2[0], p2[1]), fill=128, width=2, joint="curve")
        draw.line((c[0][0], c[0][1], c[-1][0], c[-1][1]), fill=128, width=2, joint="curve")


    display(im)

In [ ]:

import torch
from collections import OrderedDict
new_state_dict = OrderedDict()

state_dict = torch.load("Impartial/pathology/model/pretrained_impartial.pth", map_location=torch.device("cpu"))[0]

for k, v in state_dict.items():
    name = k[7:] # remove `module.`
    new_state_dict[name] = v

torch.save(new_state_dict, "Impartial/pathology/model/pretrained_impartial_2.pth")

new_state_dict